In [ ]:
!pip install keras-unet-collection

     |████████████████████████████████| 67 kB 3.0 MB/s 


In [ ]:
!unzip dataset.zip

In [ ]:
!ls dataset

input  labels


In [ ]:
import os
import cv2

input_list = []
label_list = []
INPUT_PATH = '/content/dataset'

for file in os.listdir(INPUT_PATH + '/input'):
  img = cv2.imread(INPUT_PATH + '/input/' + file)
  if img is not None:
    img = cv2.resize(img, (128, 128))
    input_list.append(img)

for file in os.listdir(INPUT_PATH + '/labels'):
  img = cv2.imread(INPUT_PATH + '/labels/' + file)
  if img is not None:
    img = cv2.resize(img, (128, 128))
    label_list.append(img)

In [ ]:
import numpy as np

input_arr = np.array(input_list)
label_arr = np.array(label_list)

In [ ]:
print(input_arr.shape)
print(label_arr.shape)

(616, 256, 256, 3)
(616, 256, 256, 3)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_arr, label_arr, test_size=0.2, random_state=42)

print(X_train.shape)

(492, 256, 256, 3)


In [ ]:
from keras_unet_collection import models

In [ ]:
help(models.unet_plus_2d)

In [ ]:
model1 = models.unet_plus_2d(input_size=(256, 256, 3), 
                             filter_num=[64, 128, 256, 512], 
                             n_labels=3, 
                             stack_num_down=2, 
                             stack_num_up=2, 
                             activation='ReLU', 
                             output_activation='Softmax',
                             batch_norm=True, 
                             pool=False, 
                             unpool=False,
                             deep_supervision=True,
                             name='xnet')

----------
deep_supervision = True
names of output tensors are listed as follows ("sup0" is the shallowest supervision layer;
"final" is the final output layer):

	xnet_output_sup0_activation
	xnet_output_sup1_activation
	xnet_output_sup2_activation
	xnet_output_final_activation


In [ ]:
model1.layers[1].name

'xnet_down0_0'

In [ ]:
from tensorflow import keras

model1.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(lr=1e-3), metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
print(model1.summary())

Model: "xnet_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 xnet_down0_0 (Conv2D)          (None, 256, 256, 64  1728        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 xnet_down0_0_bn (BatchNormaliz  (None, 256, 256, 64  256        ['xnet_down0_0[0][0]']           
 ation)                         )                                                        

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 4)

In [ ]:
hist = model1.fit(X_train, y_train, verbose=2, batch_size=32, 
                  shuffle = False, epochs=10, validation_split=0.1, 
                  callbacks = [callback])

In [ ]:
print(X_train.shape, y_train.shape)

(492, 270, 480, 3) (492, 270, 480, 3)
